In [1]:
# DEFINITION OF PLOTS & OVERVIEW
def myfigures(e = None):
    def fake_e(dataItem):
        return [dataItem[:-1] + '01']
    func = fake_e if e is None else e.dataItemsCyl
    return {
        'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
        'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
        'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
        'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':func('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':func('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
        'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':func('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':func('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':func('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-80, 10], 'unit':'%'},
        ],    
    }

def overview_figure():
    return {
        'basic': [
        {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
        {'col':['runout'],'_ylim':(0,100) },
        {'col':['targetload'],'_ylim':(-4000,26000) },
        {'col':['ramprate'],'_ylim':(-5,7)},
        {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
        {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
        {'col':['synchronize'],'_ylim':(-20,400)},
        {'col':['startpreparation'],'_ylim':(-1000,800)},
        {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
        {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
        #{'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
        ]
    }

In [2]:
import warnings # default warn => SettingWithCopyWarning
import traceback
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import os, time
import pickle
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Text, Select, Tab, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output

from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();

### Statemachine Engine Analysis ©2022 Dieter Chvatal

In [3]:
# global constants
hh = '500px' # window height
qfn = './engines.pkl'

#with open('/opt/notebooks/assets/Misterious_mist.gif', 'rb') as f:
with open('./assets/Misterious_mist.gif', 'rb') as f:
    img = f.read()    
loading_bar = widgets.Image(
    value=img
)

# initialize global variables
def init_globals():
    global fleet,e,fsm,rdf,lfigures, plotdef, vset
    #fleet = None
    e = None
    lfigures = myfigures()
    plotdef, vset = cplotdef(mp, lfigures)
    fsm = None
    rdf = pd.DataFrame([])
    global query_list
    query_list = []
    if os.path.exists(qfn):
        with open(qfn, 'rb') as handle:
            query_list = pickle.load(handle)
    else:  
        query_list = ['Forsa Hartmoor','BMW Landshut']

init_globals()
    
def do_lookup(lookup):
    def sfun(x):
        #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
        return (
            (str(lookup).upper() in str(x['IB Site Name']).upper()) or \
            (str(lookup).upper() in str(x['Engine Type']).upper()) or \
            (str(lookup).upper() in str(x['Engine Version']).upper()) or \
            (str(lookup) == str(x['Design Number'])) or \
            (str(lookup) == str(x['serialNumber'])) or \
            (str(lookup) == str(x['id']))) and \
            (x['OperationalCondition'] != 'Decommissioned')

    global fleet
    fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
    fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
    ddl = [f"{x['serialNumber']}  J{x['Engine Type']} {x['Engine Version']:<4} {x['Engine ID']} {x['IB Site Name']}" for i, x in fleet.iterrows()]
    ddl = [m for m in ddl]
    return ddl

In [4]:
#########################################
# tab1
#########################################
tab1_out = widgets.Output()

@tab1_out.capture(clear_output=True)
def do_refresh(but):
    print()
    print('Please Wait ...')
    display(loading_bar)
    mp._fetch_installed_base(); # refresh local installed fleet database
    tab1_out.clear_output()
    
@tab1_out.capture(clear_output=True)
def do_sel(*args):
    sel.value = es.value
    selno.value = str(list(es.options).index(es.value))
    print()
    print(f"{len(list(es.options))} Engines found.")
    print()
    print('please select an Engine and  move to section 2.')

@tab1_out.capture(clear_output=True)
def sbcb(but):
    elst = do_lookup(tdd.value)
    es.options = elst
    # es.value = elst[0]
    if not tdd.value in query_list:
        query_list.append(tdd.value)
    #print(tdd.value, query_list)
    if os.path.exists(qfn):
        os.remove(qfn)
    with open(qfn, 'wb') as handle:
        pickle.dump(query_list, handle, protocol=5)
        
###############
# tab1 widgets
###############
tdd = widgets.Combobox(
    value='',
    # placeholder='Choose Someone',
    options=query_list,
    description='Site Name:',
    #ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width='448px'))

es = Select(
    options=['-'], 
    value='-', 
    rows=10, 
    description='Engine:', 
    disabled=False, 
    layout=widgets.Layout(width='600px'))
es.observe(do_sel, 'value')

sel = Text(
    value='-', 
    description='selected:', 
    disabled=True, 
    layout=widgets.Layout(width='400px'))

selno = Text(
    value='-', 
    description='Motor No:', 
    disabled=True, 
    layout=widgets.Layout(width='200px'))

sb = Button(description='Lookup',disabled=False, button_style='primary')
sb.on_click(sbcb)

reload_button = Button(
    disabled=False,
    button_style='',
    tooltip='Reload Installed Base Database',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    layout=Layout(width='30px !important')
)
reload_button.on_click(do_refresh)

tab1 = VBox([
        HBox([tdd,sb,reload_button]),
        es,
        HBox([sel,selno]),
        tab1_out
    ],
    layout=widgets.Layout(height=hh)
)

In [5]:
#########################################
# tab2
#########################################
tab2_out = widgets.Output()

def fsm_loadmessages(b):
    global fsm
    with tab2_out:
        tab2_out.clear_output()
        print('.. loading messages.')
        display(loading_bar)
        try:
            fsm = FSMOperator(e, p_from=t1.value, p_to=t2.value)
            tab2_out.clear_output()
            b_runfsm.button_style='primary'
            b_runfsm.disabled = False
            if not fsm.exists:
                b_loadfsm.disabled = True
                b_loadfsm.button_style = ''
            else:
                b_loadfsm.disabled = False
                b_loadfsm.button_style = 'primary'
        except Exception as err:
            tab2_out.clear_output()
            print('Error: ',str(err))

#@tab2_out.capture(clear_output=True)
def fsm_run(b):
    motor = fleet.iloc[int(selno.value)]
    with tab2_out:
        tab2_out.clear_output()
        global fsm
        if fsm is not None:
            print()
            fsm.run0(enforce=True, silent=False, debug=False)
            fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
            fsm.run2(silent = False)
            fsm.store()
            global rdf
            rdf = fsm.starts
            print()
            print(f"Starts: {rdf.shape[0]}") 
            print(f"Successful: {rdf[rdf['success'] == 'success'].shape[0]}, Failed: {rdf[rdf['success'] == 'failed'].shape[0]}, Undefined: {rdf[rdf['success'] == 'undefined'].shape[0]}")
            print(f"Starting reliability raw: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0])*100.0:3.1f}% ")
            print(f"Starting reliability: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0]-rdf[rdf['success'] == 'undefined'].shape[0])*100.0:3.1f}% ")


def fsm_load(b):
    with tab2_out:
        tab2_out.clear_output()
        print('Please Wait, loading FSM Results')
        display(loading_bar)
        fsm.restore()
        tab2_out.clear_output()
        global rdf
        rdf = fsm.starts
        print()
        if 'save_date' in fsm.results:
            print(f"{'Results Date:':>25} {fsm.results['save_date'].strftime('%d.%m.%Y')}")
        print(f"{'Time Range:':>25} {fsm.results['first_message'].strftime('%d.%m.%Y')} - {fsm.results['last_message'].strftime('%d.%m.%Y')}")
        print(f"{'Starts:':>25} {rdf.shape[0]}") 
        print(f"{'Successful:':>25} {rdf[rdf['success'] == 'success'].shape[0]}")
        print(f"{'Failed:':>25} {rdf[rdf['success'] == 'failed'].shape[0]}")
        print(f"{'Undefined:':>25} {rdf[rdf['success'] == 'undefined'].shape[0]}")
        print(f"{'Starting reliability raw:':>25} {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0])*100.0:3.1f}% ")
        print(f"{'Starting reliability:':>25} {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0]-rdf[rdf['success'] == 'undefined'].shape[0])*100.0:3.1f}% ")

###############
# tab2 widgets
###############
el = Text(
    value='-', description='selected:', disabled=True, 
    layout=Layout(width='603px'))
t1 = widgets.DatePicker( 
    value=pd.to_datetime('2022-01-01'), 
    description='From: ',disabled=False)
t2 = widgets.DatePicker( 
    value = date.today(), 
    description='To:',disabled=False)

b_loadmessages = Button(
    description='load Messages',
    disabled=False, 
    button_style='primary')
b_loadmessages.on_click(fsm_loadmessages)

b_runfsm = widgets.Button(
    description='Run FSM',
    disabled=True, 
    button_style='')
b_runfsm.on_click(fsm_run)

b_loadfsm = widgets.Button(
    description='Load FSM',
    disabled=True, 
    button_style='')
b_loadfsm.on_click(fsm_load)
tab2 = VBox([HBox([VBox([el,HBox([t1,t2])]),VBox([b_loadmessages,b_runfsm,b_loadfsm])]),tab2_out])

In [6]:
#########################################
# tab3
#########################################
tab3_out = widgets.Output()
mo = widgets.SelectMultiple( options=['undefined','OFF','MAN','AUTO'], value=['MAN','AUTO'], rows=4, description='modes: ', disabled=False)
succ = widgets.SelectMultiple( options=['success','failed','undefined'], value=['success'], rows=3, description='success: ', disabled=False)
alarm_warning = widgets.SelectMultiple( options=['-','Alarms','Warnings'], value=['-'], rows=3, description='A&W: ', disabled=False)

@tab3_out.capture(clear_output=True)
def show_overview(b):
    global rda
    rda = rdf[:].reset_index(drop='index')
    thefilter = (
        (rda['mode'].isin(mo.value)) & 
        (rda['success'].isin(succ.value)) & 
        ((rda['count_warnings'] > 0) | ('Warnings' not in alarm_warning.value)) & 
        ((rda['count_alarms'] > 0) | ('Alarms' not in alarm_warning.value))
    )
    rda = rda[thefilter].reset_index(drop='index')
    global rdb
    rdb = rda
    rde = rda #.fillna('')
    if not rde.empty:
        rde['datetime'] = pd.to_datetime(rde['starttime'])
        sdict ={'success':1, 'failed':0, 'undefined':0.5}
        rde['isuccess'] = rde.apply(lambda x: sdict[x['success']], axis=1)
        vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
        display(rde[vec].describe()
                    .style
                    .format(
                precision=0,
                na_rep='-',
                formatter={
                    'starter': "{:.1f}",
                    'idle': "{:.1f}",
                    'ramprate':"{:.2f}",
                    'runout': lambda x: f"{x:0.1f}"
                }
            ))
        dfigsize = (20,10)
        dset = overview_figure()['basic']
        dset = equal_adjust(dset, rde, do_not_adjust=[-1])
        ftitle = f"{fsm._e}"
        try:
            fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
            print()
            bokeh_show(fig)
        except Exception as err:
            print('\n','no figure to display, Error: ', str(err))

        print()
        display(rde[startstopFSM.run2filter_content]
                .style
                .hide()
                .format(
            precision=0,
            na_rep='-',
            formatter={
                'starter': "{:.1f}",
                'idle': "{:.1f}",
                'ramprate':"{:.2f}",
                'runout': lambda x: f"{x:0.1f}"
            }
        ))
    else:
        print()
        print('Empty DataFrame.')

###############
# tab3 widgets
###############
t3_button = widgets.Button(description='Overview',disabled=False, button_style='primary')
t3_button.on_click(show_overview)
tab3 = VBox([HBox([el,t3_button]), HBox([mo,succ,alarm_warning]),tab3_out])

In [7]:
#########################################
# tab4
#########################################
tab4_out = widgets.Output()
pfigsize=(18,10)


def calc_time_range(sv):
    tns = pd.to_datetime((sv['starttime'].timestamp() + time_range.value[0]/100.0 * (sv['endtime']-sv['starttime']).seconds), unit='s')
    tne = pd.to_datetime((sv['starttime'].timestamp() + time_range.value[1]/100.0 * (sv['endtime']-sv['starttime']).seconds), unit='s')
    return tns, tne


def update_fig(x=0, lfigures=lfigures, plotselection=plotdef, vset=vset, plot_range=(0,100), debug=False):
    #global rda
    global rdf
    rda = rdf[:].reset_index(drop='index')
    rda = rda[(rda['mode'].isin(mo.value) & rda['success'].isin(succ.value))].reset_index(drop='index')
    global rdb
    rdb = rda
    global vv;
    rdfs = rdf[rdf.no == x]
    if not rdfs.empty:
        startversuch = rdfs.iloc[0]
        vv = startversuch.no; 
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        with tab4_out:
            tab4_out.clear_output()
            display(loading_bar)
        print(f'Please Wait, loading data for Start No. {vv}')
        try:
            data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset, t_range=plot_range)
            data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
            tab4_out.clear_output()
            # PLotter
            ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
            fig_handles = []
            for doplot in plotselection:
                dset = lfigures[doplot]
                ltitle = f"{ftitle} | {doplot}"
                if count_columns(dset) > 12: # no legend, if too many lines.
                    fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
                else:
                    fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

                fig = FSM_add_Notations(fig, fsm, startversuch)
                disp_alarms(startversuch)
                disp_warnings(startversuch)
                fig = FSM_add_Alarms(fig, fsm, startversuch)
                fig = FSM_add_Warnings(fig, fsm, startversuch)
                fig_handles.append(bokeh_show(fig, notebook_handle=True))
            for h in fig_handles:
                push_notebook(handle=h)
            print()
            print("messages leading to state change:")    
            print("-----------------------------------")
            for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
                print(f"{i:3} {v}")
            print(f"\nall messages during start attempt No.:{vv:4d} leading to state change:")
            print("---------------------------------------------------------------------")
            for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
                print(f"{i:3} {v}")
        except Exception as err:
            tab4_out.clear_output()
            print('Error: ', str(err))
            if debug:
                print(traceback.format_exc())
    else:
        print(f"Error: Start No {x} is not in results.")

@tab4_out.capture(clear_output=True)
def show_plots(but):
    update_fig(x=sno.value, lfigures=lfigures, plotselection=plotselection.value, vset=vset, plot_range=time_range.value)

def start_info(*args):
    if not rdf.empty:
        sv = rdf.iloc[sno.value]
        global vv;
        vv = sno.value
        summary = pd.DataFrame(sv[startstopFSM.run2filter_content]).T
        r = summary.style.set_table_styles([
            {'selector':'tbody','props':'font-size:0.7rem; font-weight: bold; text-align:center; background-color: #D3D3D3; ' + \
                                     'border: 0px solid black; border-collapse: collapse; margin: 0px; padding: 0px;'},
            {'selector':'td','props':'font-size:0.7rem; text-align:center; min-width: 58px;'}]
        ).format(
            precision=0,
            na_rep='-',
            formatter={
                'starter': "{:.1f}",
                'idle': "{:.1f}",
                'ramprate':"{:.2f}",
                'runout': lambda x: f"{x:0.1f}"
            }
        ).hide().to_html()
        links = 'links to Myplant: | '
        time_new_start, time_new_end = calc_time_range(sv)
        for doplot in plotselection.options:
            ll = e.myplant_workbench_link(time_new_start, time_new_end, e.get_dataItems(dat=cvset(mp,lfigures[doplot])),doplot)
            links += f'{ll} | '
        start_table.value = links + '<br>' + r
start_info()
        
###############
# tab4 widgets
###############

sno = widgets.IntText(
    description='StartNo: ',
    layout=widgets.Layout(width='236px'))
sno_slider = widgets.IntSlider(0, 0, max(rdf.shape[0]-1,1) , 1,
    description = 'StartNo:',
    layout=widgets.Layout(width='516px'))
mylink = widgets.jslink((sno, 'value'), (sno_slider, 'value'))
sno.observe(start_info, 'value')

tshowplots = widgets.Button(description='Plots',disabled=False, button_style='primary')
tshowplots.on_click(show_plots)

plotselection = widgets.SelectMultiple( 
    options=list(myfigures().keys()), 
    value=list(myfigures().keys())[:], 
    rows=min(len(myfigures()),3), 
    disabled=False,
    description='Al & War:')
start_table = widgets.HTML()

time_range = widgets.IntRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1,
    description='Range (%):',
    disabled=False,
    #continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='500px')
)
time_range.observe(start_info,'value')

tab4 = VBox([
            HBox([
                VBox([
                    HBox([el, tshowplots]),
                    HBox([sno_slider, sno])
                ]),
                plotselection
            ]),
            time_range,
            start_table,
            tab4_out
        ]);

In [8]:
#########################################
# tabs
#########################################
tabs_out = widgets.Output()
tabs = widgets.Tab()
tabs.children = [tab1, tab2, tab3, tab4]
tabs.set_title(0,'1. select Engine')
tabs.set_title(1,'2. messages & FSM')
tabs.set_title(2,'3. Filter Results')
tabs.set_title(3,'4. Start Plots')

#@tabs_out.capture(clear_output=False)
def tabs_cb(*args):
    #tabs_out.clear_output()
    if tabs.selected_index == 0:
            init_globals()
            sel.value = '-'
            selno.value = '-'
            tdd.value = ''
            tdd.options = query_list
            #fsm.init_results()
            tab2_out.clear_output()
            tab3_out.clear_output()
            tab4_out.clear_output()
            b_runfsm.button_style=''
            b_runfsm.disabled = True
            b_loadfsm.disabled = True
            b_loadfsm.button_style = ''
            
    if tabs.selected_index > 0:
        if selno.value == '-':
            with tab1_out:
                tab1_out.clear_output()
                print()
                print('Please select an Engine first.')
            tabs.selected_index = 0
        else:
            global e
            if e is None:
                with tab2_out:
                    tab2_out.clear_output()
                    print('Please Wait ...')
                    display(loading_bar)
                    print('contacting MyPlant ...')
                    e=Engine.from_fleet(mp, fleet.iloc[int(selno.value)])
                    tab2_out.clear_output()
            global lfigures, plotdef, vset
            lfigures = myfigures(e)
            plotdef, vset = cplotdef(mp, lfigures)
            el.value = sel.value
            t1.value = pd.to_datetime(e['Commissioning Date'])
        if selno.value != '-' and ((fsm is None) or (len(fsm.starts) == 0)):
            with tab2_out:
                tab2_out.clear_output()
                print('Please load messages.')
            tabs.selected_index = 1
        if not rdf.empty:
            if sno_slider.max < len(rdf):
                sno_slider.max = rdf.shape[0]-1

#(str(lookup) in str(x['IB Site Name'])) or \
#(str(lookup) in str(x['Engine Type'])) or \
#(str(lookup) in str(x['Engine Version'])) or \
#(str(lookup) == str(x['Design Number'])) or \
#(str(lookup) == str(x['serialNumber'])) or \
#(str(lookup) == str(x['id']))) and \

with tab1_out:
    tab1_out.clear_output()
    print()
    print('Please Lookup:')
    print(' - Site Name')
    print(' - Engine Type')
    print(' - Engine Version')
    print(' - Design Number')
    print(' - serialNumber')
    print(' - assetNumber')
tabs.observe(tabs_cb, 'selected_index')
display(VBox([tabs,tabs_out]))

In [9]:
if e is not None:
    mfn = e._fname + '_messages.txt'
    #fsm.save_messages(mfn)
    print(mfn)

In [10]:
#startversuch = rdf.iloc[0]; pre_period=5*60; post_period=21*60; t_range=(0,100)
#tns = pd.to_datetime((startversuch['starttime'].timestamp() - pre_period + t_range[0]/100.0 * ((startversuch['endtime']  - startversuch['starttime']).seconds + pre_period + post_period)), unit='s')
#tne = pd.to_datetime((startversuch['starttime'].timestamp() - pre_period + t_range[1]/100.0 * ((startversuch['endtime']  - startversuch['starttime']).seconds + pre_period + post_period)), unit='s')
#t0 = tns.timestamp() * 1e3
#t1 = tne.timestamp() * 1e3f
#t0, t1

In [13]:
rdg = fsm.starts
thefilter = (
    (rdg['mode'].isin(mo.value)) & 
    (rdg['success'].isin(succ.value)) & 
    ((rdg['count_warnings'] > 0) | ('Warnings' not in alarm_warning.value)) & 
    ((rdg['count_alarms'] > 0) | ('Alarms' not in alarm_warning.value))
)
rdg = rdg[thefilter].reset_index(drop='index')
r2res = rdg[['starttime','no','ramprate','targetload','tmax','spread_at_tmax','power_at_tmax','dpmin','p_at_dpmin','t_at_dpmin']]
r2res;

In [14]:
r2res.describe()

,no,ramprate,targetload,tmax,spread_at_tmax,power_at_tmax,dpmin,p_at_dpmin,t_at_dpmin
count,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000
mean,100.848000,0.916998,4531.660286,606.196632,28.961784,2649.641544,13.804074,1031.270030,22.868706
std,54.932812,0.297025,45.210215,20.104846,24.405990,1377.378968,5.836053,12.680136,4.700607
min,14.000000,0.547655,4365.000000,579.000000,5.000000,865.246000,7.000000,985.792000,8.700000
25%,52.000000,0.701642,4499.933000,590.000000,9.979000,1070.131000,8.366000,1024.797000,22.200000
50%,99.000000,0.781962,4524.870000,601.000000,17.000000,2278.447000,13.000000,1031.906000,24.087200
75%,161.000000,1.144523,4554.131712,621.869000,48.097000,3972.366000,16.000000,1040.395000,25.528000
max,194.000000,2.375721,4642.336000,652.875000,136.565000,4569.925000,30.000000,1054.256000,30.500000


In [15]:
r2res['datetime'] = pd.to_datetime(r2res['starttime'])
dr2set = [
        {'col':['ramprate'],'_ylim': [0, 2500], '_color':'blue', 'unit':'%/s'},
        {'col':['targetload'],'_ylim': [0, 3], '_color':'rgba(255,165,0,0.4)', 'unit':'kW'},
        {'col':['tmax'],'_ylim': [0, 3], '_color':'rgba(255,0,0,0.4)', 'unit':'°C'},
        {'col':['spread_at_tmax'],'_ylim': [0, 3], '_color':'rgba(255,0,0,0.4)', 'unit':'°C'},
        {'col':['power_at_tmax'],'_ylim': [0, 3], '_color':'rgba(255,0,0,0.4)', 'unit':'°C'},
        {'col':['dpmin'],'_ylim': [0, 3], '_color':'rgba(255,0,255,0.4)', 'unit':'mbar'},
        {'col':['p_at_dpmin'],'_ylim': [0, 3], '_color':'rgba(255,0,255,0.4)', 'unit':'mbar'},
        {'col':['t_at_dpmin'],'_ylim': [0, 3], '_color':'rgba(0,255,0,0.4)', 'unit':'°C'}
        ]

dr2set = equal_adjust(dr2set, r2res, do_not_adjust=[], minfactor=0.8, maxfactor=2)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(r2res, dr2set, style='both', figsize=(14,8) ,title=ftitle);
bokeh_show(fig)

In [16]:
dr2set2 = [
        {'col':['targetload'],'ylim': [4200, 4800], 'color':'red', 'unit':'kW'},
        {'col':['dpmin'],'ylim': [0, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['p_at_dpmin'],'ylim': [700, 1200], 'color':'brown', 'unit':'mbar'},
        {'col':['t_at_dpmin'],'ylim': [0, 100], 'color':'purple', 'unit':'°C'}
        ]
dr2set2 = equal_adjust(dr2set2, r2res, do_not_adjust=[3], minfactor=0.95, maxfactor=1.2)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(r2res, dr2set2, style='both', figsize=(14,8) ,title=ftitle);
bokeh_show(fig)

In [17]:
from bokeh.models import Label, Text, Span

dr2set2 = [
        {'col':['targetload'],'ylim': [4100, 4700], 'color':'red', 'unit':'kW'},
        {'col':['dpmin'],'ylim': [0, 100], 'color':'blue', 'unit':'mbar'},
        #{'col':['p_at_dpmin'],'ylim': [800, 1300], 'color':'dodgerblue', 'unit':'mbar'},
        #{'col':['t_at_dpmin'],'ylim': [0, 100], 'color':'red', 'unit':'°C'},
        {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.05)'] },
        ]
ntitle = ftitle + ' | Load at Start vs Tecjet delta p'
#dr2set2 = equal_adjust(dr2set2, r2res, do_not_adjust=[])
fig = dbokeh_chart(r2res, dr2set2, x='t_at_dpmin', style='circle', figsize=(14,8) ,title=ntitle);
fig.add_layout(Span(location=fsm._e['Power_PowerNominal'],
        dimension='width',x_range_name='default', y_range_name='0',line_color='red', line_dash='dashed', line_alpha=0.6))
fig.add_layout(Span(location=20.0,
        dimension='width',x_range_name='default', y_range_name='1',line_color='blue', line_dash='dashed', line_alpha=0.6))

#fig = dbokeh_chart(r2res, dr2set2, x='targetload', style='circle', figsize=(14,8) ,title=ftitle);

bokeh_show(fig)

In [18]:
from bokeh.models import Label, Text, Span
r2res['bmep'] = r2res.apply(lambda x: fsm._e._calc_BMEP(x['targetload'], 1500.0), axis=1)

dr2set2 = [
        #{'col':['targetload'],'ylim': [4100, 4700], 'color':'red', 'unit':'kW'},
        {'col':['bmep'],'ylim': [22, 26], 'color':'red', 'unit':'bar'},
        {'col':['dpmin'],'ylim': [0, 100], 'color':'blue', 'unit':'mbar'},
        #{'col':['p_at_dpmin'],'ylim': [800, 1300], 'color':'dodgerblue', 'unit':'mbar'},
        #{'col':['t_at_dpmin'],'ylim': [0, 100], 'color':'red', 'unit':'°C'},
        {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.05)'] },
        ]
ntitle = ftitle + ' | BMEP at Start vs Tecjet delta p'
#dr2set2 = equal_adjust(dr2set2, r2res, do_not_adjust=[])
fig = dbokeh_chart(r2res, dr2set2, x='t_at_dpmin', style='circle', figsize=(14,8) ,title=ntitle);
#fig.add_layout(Span(location=fsm._e['Power_PowerNominal'],
#        dimension='width',x_range_name='default', y_range_name='0',line_color='red', line_dash='dashed', line_alpha=0.6))
fig.add_layout(Span(location=24.5,
        dimension='width',x_range_name='default', y_range_name='0',line_color='red', line_dash='dashed', line_alpha=0.6))

fig.add_layout(Span(location=20.0,
        dimension='width',x_range_name='default', y_range_name='1',line_color='blue', line_dash='dashed', line_alpha=0.6))

#fig = dbokeh_chart(r2res, dr2set2, x='targetload', style='circle', figsize=(14,8) ,title=ftitle);

bokeh_show(fig)

In [20]:
#_, vset = cplotdef(mp, lfigures)
#update_fig(lfigures=, ['tecjet'], vset, x=47, plot_range=(0,100), debug=True)
#update_fig(x=47, lfigures=lfigures, plotselection=['tecjet'], vset=vset, plot_range=(0,100), debug=True)
update_fig(x=47, plotselection=['tecjet'])